In [22]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import MDS
import altair as alt
import numpy as np
pd.set_option('display.max_rows', None)

In [23]:
# List all files and directories in the current directory
all_files = os.listdir('.')

# Filter the list to get only CSV files
csv_files = [f for f in all_files if f.endswith('.csv')]

# Print the list of CSV files
print("Lista dei file CSV nella cartella corrente:")
for csv in csv_files:
    print(csv)

# For the user response, I'll also check if the list is empty
if not csv_files:
    print("\nNessun file CSV trovato nella cartella corrente.")

Lista dei file CSV nella cartella corrente:
team_inverse_frequency_vectors.csv
Wolverhampton_Wanderers.csv
Crystal_Palace.csv
Aston_Villa.csv
Ipswich_Town.csv
Fulham.csv
Liverpool.csv
Newcastle_United.csv
Premier_League_24_25.csv
Southampton.csv
team_frequency_vectors.csv
Chelsea.csv
Leicester_City.csv
Brentford.csv
unique_event_sequences_vocabulary.csv
Brighton_Hove_Albion.csv
Manchester_City.csv
West_Ham_United.csv
Tottenham_Hotspur.csv
Nottingham_Forest.csv
AFC_Bournemouth.csv
Arsenal.csv
Manchester_United.csv
team_one_hot_vectors.csv
Everton.csv


In [24]:
# Lista di tutti i file forniti dall'utente
file_list = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]
# Insieme per memorizzare tutte le parole uniche
unique_words_vocabulary = set()
files_processed = 0
files_not_found = []

print(f"Tentativo di processare {len(file_list)} file CSV...")

for file_name in file_list:
    try:
        # Leggi il file CSV
        df = pd.read_csv(file_name)
        
        # Controlla se la colonna 'words' esiste
        if 'words' in df.columns:
            # Assumendo la nuova interpretazione: ogni valore della colonna 'words' è una singola "parola" (sequenza di eventi).
            
            # 1. Riempi eventuali valori mancanti (NaN) con una stringa vuota
            # 2. Converti in set per ottenere valori unici in questo file
            words_series = df['words'].fillna('').astype(str).str.strip()
            # Filtra le stringhe vuote risultanti
            current_file_words = set(words_series[words_series != ''])
            
            # Aggiorna l'insieme globale con le parole uniche di questo file
            unique_words_vocabulary.update(current_file_words)
            
            files_processed += 1
            
        else:
            print(f"Attenzione: La colonna 'words' non è stata trovata nel file {file_name}. File saltato.")

    except FileNotFoundError:
        files_not_found.append(file_name)
    except Exception as e:
        # Cattura altri possibili errori (es. malformazione del CSV)
        print(f"Errore durante l'elaborazione del file {file_name}: {e}. File saltato.")

print("\n--- Risultato dell'elaborazione ---")
print(f"File processati con successo: {files_processed} su {len(file_list)}")
print(f"File non trovati: {len(files_not_found)}")

if files_not_found:
    print(f"I seguenti file non sono stati trovati nel tuo ambiente di lavoro e non sono stati inclusi nel vocabolario: {', '.join(files_not_found)}")

# Converti l'insieme in una lista per la presentazione
final_vocabulary_list = sorted(list(unique_words_vocabulary))

print(f"\nNumero totale di 'parole' uniche trovate: {len(final_vocabulary_list)}")

# Mostra un campione delle prime 5 parole più lunghe e delle prime 5 parole più corte, se possibile
if len(final_vocabulary_list) > 0:
    # Ordina per lunghezza e poi alfabeticamente per avere un campione stabile
    sorted_by_length = sorted(final_vocabulary_list, key=lambda x: (len(x), x))
    
    sample_short = sorted_by_length[:5]
    sample_long = sorted_by_length[-5:]
    
    print("\nEsempio di 'parole' uniche (sequenze di eventi):")
    
    print("\n5 'parole' più corte:")
    for word in sample_short:
        print(f"- {word}")
        
    if len(final_vocabulary_list) > 5:
        print("\n5 'parole' più lunghe:")
        for word in reversed(sample_long): # Le prendiamo al contrario per mostrare le più lunghe in alto
            print(f"- {word[:100]}...") # Tronca per non stampare righe enormi

Tentativo di processare 20 file CSV...

--- Risultato dell'elaborazione ---
File processati con successo: 20 su 20
File non trovati: 0

Numero totale di 'parole' uniche trovate: 8690

Esempio di 'parole' uniche (sequenze di eventi):

5 'parole' più corte:
- Duel
- Pass
- Shot
- 50/50
- Block

5 'parole' più lunghe:
- Pass-Ball Receipt-Carry-Pass-Ball Receipt-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball ...
- Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass...
- Goal Keeper-Ball Recovery-Pass-Ball Receipt-Pass-Ball Receipt-Pass-Ball Receipt-Carry-Pass-Ball Rece...
- Pass-Ball Receipt-Carry-Pass-Ball Receipt-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball ...
- Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass...


In [25]:
# Salviamo la lista in un file CSV per una facile ri-utilizzabilità futura se necessario
output_df = pd.DataFrame(final_vocabulary_list, columns=['unique_event_sequences'])
output_file = 'unique_event_sequences_vocabulary.csv'
output_df.to_csv(output_file, index=False)
print(f"\nIl vocabolario delle sequenze di eventi uniche è stato salvato nel file: {output_file}")


Il vocabolario delle sequenze di eventi uniche è stato salvato nel file: unique_event_sequences_vocabulary.csv


In [26]:
# 1. Lista dei file forniti dall'utente
team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]

vocabulary_file = 'unique_event_sequences_vocabulary.csv'
team_vectors = []
files_not_found = []

try:
    # 2. Carica il vocabolario globale
    vocab_df = pd.read_csv(vocabulary_file)
    global_vocabulary = sorted(vocab_df['unique_event_sequences'].tolist())
    vocab_size = len(global_vocabulary)
    
    # Crea una mappatura dalla parola all'indice
    word_to_index = {word: i for i, word in enumerate(global_vocabulary)}

    print(f"Vocabolario caricato con successo dal file: {vocabulary_file}")
    print(f"Dimensione del vocabolario disponibile (non 9337): {vocab_size}")
    
    if vocab_size != 9337:
        print(f"NOTA: La dimensione del vocabolario trovato è {vocab_size}, non 9337 come richiesto. Verranno generati vettori di questa lunghezza.")

    # 3. Iterazione attraverso i file delle squadre
    for file_name in team_files:
        try:
            # Estrarre il nome della squadra dal nome del file
            team_name = file_name.replace('.csv', '')
            
            # Leggi il file CSV
            df = pd.read_csv(file_name)
            
            # Controlla se la colonna 'words' esiste
            if 'words' in df.columns:
                # Ottieni le 'parole' uniche (sequenze di eventi) per questa squadra
                current_team_words = set(df['words'].fillna('').astype(str).str.strip().tolist())
                current_team_words.discard('') # Rimuovi stringhe vuote
                
                # 4. Creazione del one-hot vector
                # Inizializza il vettore a zeri
                one_hot_vector = np.zeros(vocab_size, dtype=int)
                
                # Imposta a 1 le posizioni delle parole presenti
                for word in current_team_words:
                    if word in word_to_index:
                        index = word_to_index[word]
                        one_hot_vector[index] = 1
                
                # Aggiungi il risultato alla lista (nome squadra + vettore)
                team_vectors.append({'Team': team_name, 'Vector': one_hot_vector})
                print(f"Vettore one-hot creato per {team_name} (lunghezza {vocab_size}).")
                
            else:
                print(f"Attenzione: La colonna 'words' non è stata trovata nel file {file_name}. Squadra saltata.")

        except FileNotFoundError:
            files_not_found.append(file_name)
        except Exception as e:
            print(f"Errore durante l'elaborazione del file {file_name}: {e}. Squadra saltata.")

except FileNotFoundError:
    print(f"Errore: File del vocabolario non trovato ({vocabulary_file}). Eseguire il passaggio precedente per crearlo.")
    team_vectors = None
except Exception as e:
    print(f"Errore durante il caricamento del vocabolario: {e}")
    team_vectors = None

if team_vectors:
    # 5. Salva i risultati
    # Costruisci il DataFrame finale
    
    # Crea una lista di righe, dove ogni riga è il nome della squadra seguito dagli elementi del vettore
    data_for_df = []
    
    # Nomi delle colonne: 'Team' + le parole del vocabolario
    column_names = ['Team'] + global_vocabulary
    
    for item in team_vectors:
        row = [item['Team']] + item['Vector'].tolist()
        data_for_df.append(row)
    
    final_df = pd.DataFrame(data_for_df, columns=column_names)
    
    output_file = 'team_one_hot_vectors.csv'
    final_df.to_csv(output_file, index=False)
    
    print("\n--- Risultato ---")
    print(f"Creazione dei vettori completata per {len(team_vectors)} squadra/e.")
    print(f"I vettori one-hot (lunghezza {vocab_size}) sono stati salvati nel file: {output_file}")
    
if files_not_found:
    print(f"\nAVVISO: I seguenti file non sono stati trovati e non sono stati inclusi nel risultato: {', '.join(files_not_found)}")

Vocabolario caricato con successo dal file: unique_event_sequences_vocabulary.csv
Dimensione del vocabolario disponibile (non 9337): 8690
NOTA: La dimensione del vocabolario trovato è 8690, non 9337 come richiesto. Verranno generati vettori di questa lunghezza.
Vettore one-hot creato per AFC_Bournemouth (lunghezza 8690).
Vettore one-hot creato per Arsenal (lunghezza 8690).
Vettore one-hot creato per Aston_Villa (lunghezza 8690).
Vettore one-hot creato per Brentford (lunghezza 8690).
Vettore one-hot creato per Brighton_Hove_Albion (lunghezza 8690).
Vettore one-hot creato per Chelsea (lunghezza 8690).
Vettore one-hot creato per Crystal_Palace (lunghezza 8690).
Vettore one-hot creato per Everton (lunghezza 8690).
Vettore one-hot creato per Fulham (lunghezza 8690).
Vettore one-hot creato per Ipswich_Town (lunghezza 8690).
Vettore one-hot creato per Leicester_City (lunghezza 8690).
Vettore one-hot creato per Liverpool (lunghezza 8690).
Vettore one-hot creato per Manchester_City (lunghezza 8

In [27]:
# 1. Load the data
try:
    df_one_hot = pd.read_csv('team_one_hot_vectors.csv')
except FileNotFoundError:
    print("Errore: File 'team_one_hot_vectors.csv' non trovato. Si prega di eseguire il passaggio precedente per crearlo.")
    raise

# 2. Separate features
teams = df_one_hot['Team']
X = df_one_hot.drop('Team', axis=1)

n_samples = X.shape[0]
n_components = 2

# Inizializza il DataFrame per i risultati PCA
X_pca = pd.DataFrame(columns=[f'PC{i+1}' for i in range(n_components)])

if n_samples < n_components:
    print(f"AVVISO: PCA in 2 dimensioni richiede almeno 2 campioni. Trovati solo {n_samples} campioni (squadre).")
    
    if n_samples == 1:
        # Per un singolo campione, la PCA non è significativa. Si usa [0.0, 0.0] per il plotting.
        print("Il vettore contiene solo una squadra. Il risultato della PCA sarà un singolo punto all'origine per la visualizzazione.")
        X_pca = pd.DataFrame([[0.0, 0.0]], columns=['PC1', 'PC2'])
    else:
        # Caso teorico con 1 < n_samples < 2
        print("Impossibile eseguire la PCA a 2 dimensioni con un numero insufficiente di campioni.")
        raise ValueError("Insufficient number of samples for 2 components PCA.")
        
else:
    # 3. Perform PCA
    pca = PCA(n_components=n_components)
    X_pca_array = pca.fit_transform(X)
    X_pca = pd.DataFrame(X_pca_array, columns=[f'PC{i+1}' for i in range(n_components)])
    print(f"PCA completata. Varianza spiegata dal PC1: {pca.explained_variance_ratio_[0]:.2f}, PC2: {pca.explained_variance_ratio_[1]:.2f}")


# 4. Create a DataFrame for visualization
df_plot = pd.concat([teams.reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)

# 5. Visualize (using Altair)
chart = alt.Chart(df_plot).mark_circle(size=100).encode(
    x=alt.X('PC1', title='Componente Principale 1'),
    y=alt.Y('PC2', title='Componente Principale 2'),
    color=alt.Color('Team', legend=alt.Legend(title="Squadra")), # Colore basato sulla squadra
    tooltip=['Team', alt.Tooltip('PC1', format='.4f'), alt.Tooltip('PC2', format='.4f')]
).properties(
    title=f"PCA dei Vettori One-Hot delle Squadre (N={n_samples})",
    width=700, # Aumento la larghezza
    height=500 # Aumento l'altezza
)

# Aggiungi le etichette delle squadre
text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=7 # Sposta leggermente il testo a destra
).encode(
    text='Team',
    color=alt.value('black') # Etichette nere
)

# Combina punti e testo, e rendi il grafico interattivo
final_chart = (chart + text).interactive()
final_chart

PCA completata. Varianza spiegata dal PC1: 0.06, PC2: 0.06


alt.LayerChart(...)

In [28]:
team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]


vocabulary_file = 'unique_event_sequences_vocabulary.csv'
team_vectors = []
files_not_found = []

try:
    # 1. Carica il vocabolario globale
    vocab_df = pd.read_csv(vocabulary_file)
    global_vocabulary = sorted(vocab_df['unique_event_sequences'].tolist())
    vocab_size = len(global_vocabulary)
    
    # Crea una mappatura dalla parola all'indice
    word_to_index = {word: i for i, word in enumerate(global_vocabulary)}

    print(f"Vocabolario caricato con successo (Dimensione: {vocab_size})")

    # 2. Iterazione attraverso i file delle squadre
    for file_name in team_files:
        try:
            team_name = file_name.replace('.csv', '')
            
            # Leggi il file CSV
            df = pd.read_csv(file_name)
            
            if 'words' in df.columns:
                # Calcola la frequenza di ogni 'parola' (sequenza di eventi)
                # Pulisce i dati prima di contare
                words_series = df['words'].fillna('').astype(str).str.strip()
                # value_counts() calcola la frequenza
                word_counts = words_series[words_series != ''].value_counts()
                
                # 3. Creazione del Frequency Vector
                # Inizializza il vettore a zeri
                frequency_vector = np.zeros(vocab_size, dtype=int)
                
                # Imposta la frequenza per le parole presenti
                for word, count in word_counts.items():
                    if word in word_to_index:
                        index = word_to_index[word]
                        # Assegna la frequenza (count) anziché 1
                        frequency_vector[index] = count
                
                # Aggiungi il risultato alla lista (nome squadra + vettore)
                team_vectors.append({'Team': team_name, 'Vector': frequency_vector})
                print(f"Vettore di frequenza creato per {team_name} (lunghezza {vocab_size}).")
                
            else:
                print(f"Attenzione: La colonna 'words' non è stata trovata nel file {file_name}. Squadra saltata.")

        except FileNotFoundError:
            files_not_found.append(file_name)
        except Exception as e:
            print(f"Errore durante l'elaborazione del file {file_name}: {e}. Squadra saltata.")

except FileNotFoundError:
    print(f"Errore: File del vocabolario non trovato ({vocabulary_file}). Eseguire il passaggio precedente per crearlo.")
    team_vectors = None
except Exception as e:
    print(f"Errore durante il caricamento del vocabolario: {e}")
    team_vectors = None

if team_vectors:
    # 4. Salva i risultati
    data_for_df = []
    column_names = ['Team'] + global_vocabulary
    
    for item in team_vectors:
        row = [item['Team']] + item['Vector'].tolist()
        data_for_df.append(row)
    
    final_df = pd.DataFrame(data_for_df, columns=column_names)
    
    output_file = 'team_frequency_vectors.csv'
    final_df.to_csv(output_file, index=False)
    
    print("\n--- Risultato ---")
    print(f"Creazione dei vettori di frequenza completata per {len(team_vectors)} squadra/e.")
    print(f"I vettori di frequenza (lunghezza {vocab_size}) sono stati salvati nel file: {output_file}")

Vocabolario caricato con successo (Dimensione: 8690)
Vettore di frequenza creato per AFC_Bournemouth (lunghezza 8690).
Vettore di frequenza creato per Arsenal (lunghezza 8690).
Vettore di frequenza creato per Aston_Villa (lunghezza 8690).
Vettore di frequenza creato per Brentford (lunghezza 8690).
Vettore di frequenza creato per Brighton_Hove_Albion (lunghezza 8690).
Vettore di frequenza creato per Chelsea (lunghezza 8690).
Vettore di frequenza creato per Crystal_Palace (lunghezza 8690).
Vettore di frequenza creato per Everton (lunghezza 8690).
Vettore di frequenza creato per Fulham (lunghezza 8690).
Vettore di frequenza creato per Ipswich_Town (lunghezza 8690).
Vettore di frequenza creato per Leicester_City (lunghezza 8690).
Vettore di frequenza creato per Liverpool (lunghezza 8690).
Vettore di frequenza creato per Manchester_City (lunghezza 8690).
Vettore di frequenza creato per Manchester_United (lunghezza 8690).
Vettore di frequenza creato per Newcastle_United (lunghezza 8690).
Vet

In [29]:
# 1. Load the data: MODIFICA IL NOME DEL FILE
try:
    df_freq_vectors = pd.read_csv('team_frequency_vectors.csv')
except FileNotFoundError:
    print("Errore: File 'team_frequency_vectors.csv' non trovato. Si prega di eseguire il passaggio precedente per crearlo.")
    raise

# 2. Separate features (usa il nuovo DataFrame: df_freq_vectors)
teams = df_freq_vectors['Team']
X = df_freq_vectors.drop('Team', axis=1)

n_samples = X.shape[0]
n_components = 2

# Inizializza il DataFrame per i risultati PCA
X_pca = pd.DataFrame(columns=[f'PC{i+1}' for i in range(n_components)])

if n_samples < n_components:
    print(f"AVVISO: PCA in 2 dimensioni richiede almeno 2 campioni. Trovati solo {n_samples} campioni (squadre).")
    
    if n_samples == 1:
        # Per un singolo campione, la PCA non è significativa. Si usa [0.0, 0.0] per il plotting.
        print("Il vettore contiene solo una squadra. Il risultato della PCA sarà un singolo punto all'origine per la visualizzazione.")
        X_pca = pd.DataFrame([[0.0, 0.0]], columns=['PC1', 'PC2'])
    else:
        # Caso teorico con 1 < n_samples < 2
        print("Impossibile eseguire la PCA a 2 dimensioni con un numero insufficiente di campioni.")
        raise ValueError("Insufficient number of samples for 2 components PCA.")
        
else:
    # 3. Perform PCA
    pca = PCA(n_components=n_components)
    X_pca_array = pca.fit_transform(X)
    X_pca = pd.DataFrame(X_pca_array, columns=[f'PC{i+1}' for i in range(n_components)])
    print(f"PCA completata. Varianza spiegata dal PC1: {pca.explained_variance_ratio_[0]:.2f}, PC2: {pca.explained_variance_ratio_[1]:.2f}")


# 4. Create a DataFrame for visualization
df_plot = pd.concat([teams.reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)

# 5. Visualize (using Altair)
chart = alt.Chart(df_plot).mark_circle(size=100).encode(
    x=alt.X('PC1', title='Componente Principale 1'),
    y=alt.Y('PC2', title='Componente Principale 2'),
    color=alt.Color('Team', legend=alt.Legend(title="Squadra")), # Colore basato sulla squadra
    tooltip=['Team', alt.Tooltip('PC1', format='.4f'), alt.Tooltip('PC2', format='.4f')]
).properties(
    # MODIFICA IL TITOLO
    title=f"PCA dei Vettori di Frequenza delle Squadre (N={n_samples})",
    width=700, 
    height=500 
)

# Aggiungi le etichette delle squadre
text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=7 # Sposta leggermente il testo a destra
).encode(
    text='Team',
    color=alt.value('black') # Etichette nere
)

# Combina punti e testo, e rendi il grafico interattivo
final_chart = (chart + text).interactive()
final_chart

PCA completata. Varianza spiegata dal PC1: 0.85, PC2: 0.07


alt.LayerChart(...)

In [30]:
# --- CONFIGURAZIONI ---
team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]

vocabulary_file = 'unique_event_sequences_vocabulary.csv'
team_vectors = []
files_not_found = []
output_vector_file = 'team_inverse_frequency_vectors.csv'
N_COMPONENTS = 2

# --- PARTE 1: CREAZIONE DEI VETTORI DI INVERSO FREQUENZA (1/f) ---

try:
    # 1. Carica il vocabolario globale
    vocab_df = pd.read_csv(vocabulary_file)
    global_vocabulary = sorted(vocab_df['unique_event_sequences'].tolist())
    vocab_size = len(global_vocabulary)
    
    word_to_index = {word: i for i, word in enumerate(global_vocabulary)}

    print(f"Vocabolario caricato con successo (Dimensione: {vocab_size})")

    # 2. Iterazione attraverso i file delle squadre
    for file_name in team_files:
        try:
            team_name = file_name.replace('.csv', '')
            df = pd.read_csv(file_name)
            
            if 'words' in df.columns:
                # Calcola la frequenza di ogni 'parola' (sequenza di eventi)
                words_series = df['words'].fillna('').astype(str).str.strip()
                word_counts = words_series[words_series != ''].value_counts()
                
                # 3. Creazione del Vettore Raw Count
                frequency_vector = np.zeros(vocab_size, dtype=int)
                
                for word, count in word_counts.items():
                    if word in word_to_index:
                        index = word_to_index[word]
                        frequency_vector[index] = count
                
                # 4. TRASFORMAZIONE IN INVERSO FREQUENZA (1/f)
                inverse_frequency_vector = frequency_vector.astype(float)
                
                # Trova gli indici dove la frequenza è maggiore di zero
                non_zero_indices = inverse_frequency_vector > 0
                
                # Applica 1/f solo dove la frequenza non è zero
                # np.reciprocal(x) calcola 1/x
                inverse_frequency_vector[non_zero_indices] = np.reciprocal(inverse_frequency_vector[non_zero_indices])
                
                # Aggiungi il risultato alla lista (nome squadra + vettore)
                team_vectors.append({'Team': team_name, 'Vector': inverse_frequency_vector})
                print(f"Vettore di inverso frequenza creato per {team_name} (lunghezza {vocab_size}).")
                
            else:
                print(f"Attenzione: La colonna 'words' non è stata trovata nel file {file_name}. Squadra saltata.")

        except FileNotFoundError:
            files_not_found.append(file_name)
        except Exception as e:
            print(f"Errore durante l'elaborazione del file {file_name}: {e}. Squadra saltata.")

except FileNotFoundError:
    print(f"Errore: File del vocabolario non trovato ({vocabulary_file}). Eseguire il passaggio precedente per crearlo.")
    team_vectors = None
except Exception as e:
    print(f"Errore durante il caricamento del vocabolario: {e}")
    team_vectors = None

if team_vectors:
    # 5. Salva i risultati
    data_for_df = []
    column_names = ['Team'] + global_vocabulary
    
    for item in team_vectors:
        row = [item['Team']] + item['Vector'].tolist()
        data_for_df.append(row)
    
    final_df = pd.DataFrame(data_for_df, columns=column_names)
    final_df.to_csv(output_vector_file, index=False)
    
    print("\n--- Risultato: Vettori di Inverso Frequenza ---")
    print(f"Creazione dei vettori completata per {len(team_vectors)} squadra/e.")
    print(f"I vettori di inverso frequenza sono stati salvati nel file: {output_vector_file}")
    
if files_not_found:
    print(f"\nAVVISO: I seguenti file non sono stati trovati: {', '.join(files_not_found)}")

# --- PARTE 2: PCA E VISUALIZZAZIONE ---

if team_vectors:
    # 1. Load the data
    df_inv_freq_vectors = pd.read_csv(output_vector_file)

    # 2. Separate features
    teams = df_inv_freq_vectors['Team']
    X = df_inv_freq_vectors.drop('Team', axis=1)

    n_samples = X.shape[0]

    # Inizializza il DataFrame per i risultati PCA
    X_pca = pd.DataFrame(columns=[f'PC{i+1}' for i in range(N_COMPONENTS)])
    explained_variance_ratio = [0.0, 0.0]

    if n_samples < N_COMPONENTS:
        print(f"\nAVVISO: PCA in {N_COMPONENTS} dimensioni richiede almeno {N_COMPONENTS} campioni. Trovati solo {n_samples} campioni.")
        
        if n_samples == 1:
            X_pca = pd.DataFrame([[0.0, 0.0]], columns=['PC1', 'PC2'])
            print("Il risultato della PCA sarà un singolo punto all'origine.")
            
    else:
        # 3. Perform PCA
        pca = PCA(n_components=N_COMPONENTS)
        X_pca_array = pca.fit_transform(X)
        X_pca = pd.DataFrame(X_pca_array, columns=[f'PC{i+1}' for i in range(N_COMPONENTS)])
        explained_variance_ratio = pca.explained_variance_ratio_
        print(f"PCA completata. Varianza spiegata dal PC1: {explained_variance_ratio[0]:.2f}, PC2: {explained_variance_ratio[1]:.2f}")


    # 4. Create a DataFrame for visualization
    df_plot = pd.concat([teams.reset_index(drop=True), X_pca.reset_index(drop=True)], axis=1)

    # 5. Visualize (using Altair)
    chart = alt.Chart(df_plot).mark_circle(size=200).encode(
        x=alt.X('PC1', title='Componente Principale 1', scale=alt.Scale(zero=False)),
        y=alt.Y('PC2', title='Componente Principale 2', scale=alt.Scale(zero=False)),
        color=alt.Color('Team', legend=alt.Legend(title="Squadra")),
        tooltip=['Team', alt.Tooltip('PC1', format='.4f'), alt.Tooltip('PC2', format='.4f')]
    ).properties(
        # TITOLO AGGIORNATO
        title=f"PCA dei Vettori di Inverso Frequenza (1/f) delle Squadre (N={n_samples})",
        width=700,
        height=500
    )

    # Aggiungi le etichette delle squadre
    text = chart.mark_text(
        align='left',
        baseline='middle',
        dx=10
    ).encode(
        text='Team',
        color=alt.value('black')
    )

    # Combina punti e testo, e rendi il grafico interattivo
    final_chart = (chart + text).interactive()
    final_chart

Vocabolario caricato con successo (Dimensione: 8690)
Vettore di inverso frequenza creato per AFC_Bournemouth (lunghezza 8690).
Vettore di inverso frequenza creato per Arsenal (lunghezza 8690).
Vettore di inverso frequenza creato per Aston_Villa (lunghezza 8690).
Vettore di inverso frequenza creato per Brentford (lunghezza 8690).
Vettore di inverso frequenza creato per Brighton_Hove_Albion (lunghezza 8690).
Vettore di inverso frequenza creato per Chelsea (lunghezza 8690).
Vettore di inverso frequenza creato per Crystal_Palace (lunghezza 8690).
Vettore di inverso frequenza creato per Everton (lunghezza 8690).
Vettore di inverso frequenza creato per Fulham (lunghezza 8690).
Vettore di inverso frequenza creato per Ipswich_Town (lunghezza 8690).
Vettore di inverso frequenza creato per Leicester_City (lunghezza 8690).
Vettore di inverso frequenza creato per Liverpool (lunghezza 8690).
Vettore di inverso frequenza creato per Manchester_City (lunghezza 8690).
Vettore di inverso frequenza creat

In [31]:
final_chart

alt.LayerChart(...)

In [32]:
# 1. Configurazione: Lista dei file delle squadre
team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]

# 2. Caricamento dati e calcolo frequenze
team_data = {}
all_words = set()

for file in team_files:
    if os.path.exists(file):
        team_name = file.replace(".csv", "")
        df = pd.read_csv(file)
        # Assumendo che la colonna con le sequenze si chiami 'words'
        counts = df['words'].value_counts().to_dict()
        team_data[team_name] = counts
        all_words.update(counts.keys())

if len(team_data) < 2:
    print("Errore: Necessari almeno due file CSV validi per procedere.")
else:
    # 3. Identificazione words_with_large_gap (Scarto > 50 tra qualunque coppia)
    # Lo scarto massimo tra qualunque coppia è la differenza tra Max e Min delle frequenze
    words_with_large_gap = []
    for word in all_words:
        freqs = [team_data[t].get(word, 0) for t in team_data]
        if (max(freqs) - min(freqs)) > 50:
            words_with_large_gap.append(word)

    #print(f"Numero di parole con scarto > 50: {len(words_with_large_gap)}")
    #print("Lista parole significative:", words_with_large_gap)

    # 4. Creazione Matrice delle Frequenze (f)
    teams_list = list(team_data.keys())
    matrix_data = []
    for team in teams_list:
        row = [team_data[team].get(word, 0) for word in words_with_large_gap]
        matrix_data.append(row)

    df_matrix = pd.DataFrame(matrix_data, index=teams_list, columns=words_with_large_gap)

    # 5. Calcolo PCA
    # Standardizzazione
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_matrix)

    # PCA a 2 componenti
    pca = PCA(n_components=2)
    pca_results = pca.fit_transform(scaled_data)

    # Preparazione DataFrame per Altair
    df_plot = pd.DataFrame({
        'PC1': pca_results[:, 0],
        'PC2': pca_results[:, 1],
        'Team': teams_list
    })

    n_samples = len(words_with_large_gap)

    # 6. Visualizzazione Altair (PCA 2D)
    chart = alt.Chart(df_plot).mark_circle(size=200).encode(
        x=alt.X('PC1', title='Componente Principale 1', scale=alt.Scale(zero=False)),
        y=alt.Y('PC2', title='Componente Principale 2', scale=alt.Scale(zero=False)),
        color=alt.Color('Team', legend=alt.Legend(title="Squadra")),
        tooltip=['Team', alt.Tooltip('PC1', format='.4f'), alt.Tooltip('PC2', format='.4f')]
    ).properties(
        title=f"PCA dei Vettori di Frequenza (f) delle Squadre (N={n_samples})",
        width=700,
        height=500
    )

    # Aggiunta etichette nomi squadre
    text = chart.mark_text(
        align='left',
        baseline='middle',
        dx=10
    ).encode(
        text='Team',
        color=alt.value('black')
    )

    # Combinazione e resa interattiva
    final_pca_chart = (chart + text).interactive()
    
    # Visualizza il grafico nel notebook
    final_pca_chart.display()

alt.LayerChart(...)

In [33]:
print(len(words_with_large_gap))
words_with_large_gap

41


['Block',
 'Dribble-Carry',
 'Dribbled Past',
 'Ball Receipt-Pass',
 'Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry',
 'Dispossessed',
 'Goal Keeper',
 'Foul Committed',
 'Pass-Ball Receipt-Pass-Ball Receipt-Carry',
 'Ball Recovery-Carry-Pass-Ball Receipt-Carry',
 'Pass-Pressure-Ball Receipt',
 'Pressure',
 'Ball Receipt-Carry-Pass',
 'Ball Receipt-Carry-Pass-Ball Receipt-Carry',
 'Block-Goal Keeper',
 'Clearance',
 'Ball Receipt-Carry-Pass-Ball Receipt',
 'Duel',
 'Ball Receipt-Carry',
 'Miscontrol',
 'Pass-Ball Receipt-Carry-Pass',
 'Shot',
 'Dribble',
 'Pressure-Foul Committed',
 'Pass-Ball Receipt-Carry',
 'Ball Recovery-Carry',
 'Pass-Ball Receipt-Carry-Pass-Ball Receipt',
 'Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry',
 'Pass-Ball Receipt-Duel',
 'Pressure-Pressure',
 'Pass-Ball Receipt-Carry-Pass-Ball Receipt-Carry',
 'Pass-Ball Receipt-Pass',
 'Pass',
 'Ball Receipt-Pass-Ball R

In [34]:
team_data = {}
all_words = set()

# 2. Caricamento dati
for file in team_files:
    if os.path.exists(file):
        team_name = file.replace(".csv", "")
        df = pd.read_csv(file)
        counts = df['words'].value_counts().to_dict()
        team_data[team_name] = counts
        all_words.update(counts.keys())

if len(team_data) < 2:
    print("Errore: Necessari almeno due file CSV validi.")
else:
    # 3. Identificazione words_with_large_gap (Gap > 50)
    words_with_large_gap = []
    for word in all_words:
        freqs = [team_data[t].get(word, 0) for t in team_data]
        if (max(freqs) - min(freqs)) > 50:
            words_with_large_gap.append(word)

    print(f"Features identificate (Gap > 50): {len(words_with_large_gap)}")

    # 4. Creazione Matrice delle Frequenze (f)
    teams_list = list(team_data.keys())
    matrix_data = [[team_data[team].get(word, 0) for word in words_with_large_gap] for team in teams_list]
    df_matrix = pd.DataFrame(matrix_data, index=teams_list, columns=words_with_large_gap)

    # 5. Riduzione Dimensionale Non-Lineare (MDS)
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_matrix)

    # MDS è molto efficace per visualizzare distanze/somiglianze
    # Usiamo metric=True per preservare le distanze euclidee tra i profili di frequenza
    reducer = MDS(n_components=2, random_state=42, normalized_stress='auto')
    embedding = reducer.fit_transform(scaled_data)

    # 6. Preparazione DataFrame per Altair
    df_plot = pd.DataFrame({
        'Dim1': embedding[:, 0],
        'Dim2': embedding[:, 1],
        'Team': teams_list
    })

    n_samples = len(words_with_large_gap)

    # 7. Visualizzazione Altair
    chart = alt.Chart(df_plot).mark_circle(size=200).encode(
        x=alt.X('Dim1', title='Dimensione 1', scale=alt.Scale(zero=False)),
        y=alt.Y('Dim2', title='Dimensione 2', scale=alt.Scale(zero=False)),
        color=alt.Color('Team', legend=alt.Legend(title="Squadra")),
        tooltip=['Team', alt.Tooltip('Dim1', format='.4f'), alt.Tooltip('Dim2', format='.4f')]
    ).properties(
        title=f"MDS dei Vettori di Frequenza (f) delle Squadre (N={n_samples})",
        width=700,
        height=500
    )

    text = chart.mark_text(
        align='left',
        baseline='middle',
        dx=10
    ).encode(
        text='Team',
        color=alt.value('black')
    )

    final_chart = (chart + text).interactive()
    final_chart.display()

Features identificate (Gap > 50): 41


/opt/anaconda3/lib/python3.13/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(


alt.LayerChart(...)

In [35]:
import pandas as pd
import os
import numpy as np
import altair as alt
from gensim.models import Word2Vec
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler

# --- CONFIGURAZIONE ---
team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]


all_match_sentences = []
team_sequences = {}

# 1. PRE-PROCESSING: Ogni riga è una parola, ogni partita è una frase
for file in team_files:
    if os.path.exists(file):
        team_name = file.replace(".csv", "")
        df = pd.read_csv(file)
        
        # Raggruppiamo per match_id per mantenere l'ordine cronologico delle "parole"
        # Ogni partita diventa una lista di stringhe (le tue parole-sequenza)
        matches = df.groupby('match_id')['words'].apply(list).tolist()
        
        team_sequences[team_name] = [word for match in matches for word in match]
        all_match_sentences.extend(matches)

if not all_match_sentences:
    print("Errore: Nessun dato trovato. Controlla i file CSV.")
else:
    # 2. TRAINING WORD2VEC
    # Qui ogni "parola" è la tua intera sequenza (es: "Pass-Ball Receipt-Carry")
    model = Word2Vec(
        sentences=all_match_sentences, 
        vector_size=100,  # Dimensione del vettore semantico
        window=5,         # Quante sequenze vicine considerare nel contesto
        min_count=1,      # Considera anche le sequenze che appaiono una volta sola
        workers=4,
        seed=42
    )

    # 3. CREAZIONE VETTORI SQUADRA (Media dei vettori delle sue sequenze)
    def get_team_vector(words, model):
        # Prendiamo il vettore per ogni sequenza usata dalla squadra
        vectors = [model.wv[word] for word in words if word in model.wv]
        return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

    teams_list = list(team_sequences.keys())
    team_vectors = np.array([get_team_vector(team_sequences[team], model) for team in teams_list])

    # 4. RIDUZIONE DIMENSIONALE (MDS)
    scaler = StandardScaler()
    scaled_vectors = scaler.fit_transform(team_vectors)

    # MDS per mappare le somiglianze semantiche in 2D
    reducer = MDS(n_components=2, random_state=42, normalized_stress='auto')
    embedding = reducer.fit_transform(scaled_vectors)

    # 5. DATASET PER VISUALIZZAZIONE
    df_plot = pd.DataFrame({
        'Dim1': embedding[:, 0],
        'Dim2': embedding[:, 1],
        'Team': teams_list
    })

    # 6. VISUALIZZAZIONE ALTAIR (Stile richiesto)
    chart = alt.Chart(df_plot).mark_circle(size=200).encode(
        x=alt.X('Dim1', title='Dimensione Semantica 1', scale=alt.Scale(zero=False)),
        y=alt.Y('Dim2', title='Dimensione Semantica 2', scale=alt.Scale(zero=False)),
        color=alt.Color('Team', legend=alt.Legend(title="Squadra")),
        tooltip=['Team', alt.Tooltip('Dim1', format='.4f'), alt.Tooltip('Dim2', format='.4f')]
    ).properties(
        title="Analisi Semantica Word2Vec: Stile di Gioco per Squadra",
        width=700,
        height=500
    )

    text = chart.mark_text(
        align='left',
        baseline='middle',
        dx=10
    ).encode(
        text='Team',
        color=alt.value('black')
    )

    final_chart = (chart + text).interactive()
    final_chart.display()

/opt/anaconda3/lib/python3.13/site-packages/sklearn/manifold/_mds.py:677: FutureWarning: The default value of `n_init` will change from 4 to 1 in 1.9.
  warnings.warn(


alt.LayerChart(...)

In [36]:
# 1. Configurazione
team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]

available_files = [f for f in team_files if os.path.exists(f)]

def analyze_team_shots(file_path):
    df = pd.read_csv(file_path)
    
    # Total sequences
    total_seqs = len(df)
    
    # Shot sequences (full string contains 'Shot')
    shot_mask = df['words'].str.contains('Shot')
    shot_df = df[shot_mask].copy()
    num_shots = len(shot_df)
    
    if num_shots == 0:
        return {
            'Team': df['team_name'].iloc[0] if not df.empty else file_path,
            'Total_Sequences': total_seqs,
            'Shot_Count': 0,
            'Shot_Rate_%': 0,
            'Avg_Shot_Length': 0,
            'Signature_Shot_Sequence': "N/A",
            'Avg_K_Shot': 0,
            'Avg_Beta_Shot': 0
        }

    # Length of shot sequences
    shot_df['len'] = shot_df['words'].apply(lambda x: len(x.split('-')))
    
    # Signature (most frequent full shot sequence)
    signature = shot_df['words'].value_counts().idxmax()
    
    return {
        'Team': df['team_name'].iloc[0],
        'Total_Sequences': total_seqs,
        'Shot_Count': num_shots,
        'Shot_Rate_%': (num_shots / total_seqs) * 100,
        'Avg_Shot_Length': shot_df['len'].mean(),
        'Signature_Shot_Sequence': signature,
        'Avg_K_Shot': shot_df['K'].mean(),
        'Avg_Beta_Shot': shot_df['beta'].mean()
    }

results = []
for f in available_files:
    results.append(analyze_team_shots(f))

summary_df = pd.DataFrame(results)

In [37]:
summary_df

,Team,Total_Sequences,Shot_Count,Shot_Rate_%,Avg_Shot_Length,Signature_Shot_Sequence,Avg_K_Shot,Avg_Beta_Shot
0,AFC Bournemouth,21276,586,2.754277,3.476109,Shot,1.364450,0.755032
1,Arsenal,18396,554,3.011524,3.722022,Shot,1.377457,0.765070
2,Aston Villa,18958,486,2.563562,3.193416,Shot,1.465654,0.748718
3,Brentford,19436,442,2.274130,3.509050,Shot,1.521560,0.740135
4,Brighton & Hove Albion,22239,535,2.405684,3.200000,Shot,1.347916,0.759692
5,Chelsea,19367,596,3.077400,3.796980,Shot,1.455898,0.759635
6,Crystal Palace,20937,520,2.483641,3.342308,Shot,1.355773,0.751940
7,Everton,19832,405,2.042154,3.777778,Shot,1.411609,0.750811
8,Fulham,20017,523,2.612779,3.523901,Shot,1.477177,0.750532
9,Ipswich Town,19641,377,1.919454,3.525199,Shot,1.360126,0.751314


In [38]:
import pandas as pd
import altair as alt
import os


team_files = [
    "AFC_Bournemouth.csv",
    "Arsenal.csv",
    "Aston_Villa.csv",
    "Brentford.csv",
    "Brighton_Hove_Albion.csv",
    "Chelsea.csv",
    "Crystal_Palace.csv",
    "Everton.csv",
    "Fulham.csv",
    "Ipswich_Town.csv",
    "Leicester_City.csv",
    "Liverpool.csv",
    "Manchester_City.csv",
    "Manchester_United.csv",
    "Newcastle_United.csv",
    "Nottingham_Forest.csv",
    "Southampton.csv",
    "Tottenham_Hotspur.csv",
    "West_Ham_United.csv",
    "Wolverhampton_Wanderers.csv",
]

# 1. Raccolta dati da tutti i file disponibili
results = []
for file in team_files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        # Filtriamo le sequenze che portano al tiro
        shot_df = df[df['words'].str.contains('Shot')].copy()
        
        if not shot_df.empty:
            results.append({
                'Team': df['team_name'].iloc[0],
                'Shot_Rate_%': (len(shot_df) / len(df)) * 100,
                'Avg_K_Shot': shot_df['K'].mean()
            })

df_plot = pd.DataFrame(results)

# 2. Creazione dello Scatter Plot Interattivo
# Definiamo il grafico a bolle
chart = alt.Chart(df_plot).mark_circle(size=250, opacity=0.8).encode(
    x=alt.X('Shot_Rate_%', 
            title='Efficienza (Shot Rate %)', 
            scale=alt.Scale(zero=False)),
    y=alt.Y('Avg_K_Shot', 
            title='Complessità Tattica (Avg K)', 
            scale=alt.Scale(zero=False)),
    color=alt.Color('Team', legend=None), # Colori distinti, legenda nascosta per pulizia
    tooltip=[
        alt.Tooltip('Team', title='Squadra'),
        alt.Tooltip('Shot_Rate_%', title='Efficienza %', format='.2f'),
        alt.Tooltip('Avg_K_Shot', title='Complessità (K)', format='.4f')
    ]
).properties(
    title="Mappa del Linguaggio Offensivo: Efficienza vs Complessità",
    width=800,
    height=500
)

# 3. Aggiunta delle etichette (nomi delle squadre)
text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=12,
    fontSize=11
).encode(
    text='Team'
)

# 4. Linee medie (per identificare i 4 quadranti)
line_x = alt.Chart(pd.DataFrame({'x': [df_plot['Shot_Rate_%'].mean()]})).mark_rule(
    color='red', strokeDash=[3, 3]
).encode(x='x')

line_y = alt.Chart(pd.DataFrame({'y': [df_plot['Avg_K_Shot'].mean()]})).mark_rule(
    color='blue', strokeDash=[3, 3]
).encode(y='y')

# 5. Visualizzazione finale interattiva
final_chart = (chart + text + line_x + line_y).interactive()
final_chart.display()

alt.LayerChart(...)

In [39]:
# 1. Raccolta dati da tutti i file disponibili
results = []
for file in team_files:
    if os.path.exists(file):
        df = pd.read_csv(file)
        # Filtriamo le sequenze che portano al tiro
        shot_df = df[df['words'].str.contains('Shot')].copy()
        
        if not shot_df.empty:
            # Calcolo della lunghezza (numero di azioni) per ogni sequenza di tiro
            shot_df['len'] = shot_df['words'].apply(lambda x: len(x.split('-')))
            
            results.append({
                'Team': df['team_name'].iloc[0],
                'Avg_Shot_Length': shot_df['len'].mean(),
                'Avg_K_Shot': shot_df['K'].mean()
            })

df_plot = pd.DataFrame(results)

# 2. Creazione dello Scatter Plot Interattivo con Altair
chart = alt.Chart(df_plot).mark_circle(size=250, opacity=0.8).encode(
    x=alt.X('Avg_Shot_Length', 
            title='Lunghezza Media dell\'Azione (Avg_Shot_Length)', 
            scale=alt.Scale(zero=False)),
    y=alt.Y('Avg_K_Shot', 
            title='Complessità Tattica (Avg_K_Shot)', 
            scale=alt.Scale(zero=False)),
    color=alt.Color('Team', legend=None), # Colore diverso per ogni squadra
    tooltip=[
        alt.Tooltip('Team', title='Squadra'),
        alt.Tooltip('Avg_Shot_Length', title='Lunghezza Media', format='.2f'),
        alt.Tooltip('Avg_K_Shot', title='Complessità (K)', format='.4f')
    ]
).properties(
    title="Analisi del Linguaggio Offensivo: Elaborazione vs Varietà",
    width=800,
    height=500
)

# 3. Aggiunta dei nomi delle squadre accanto ai punti
text = chart.mark_text(
    align='left',
    baseline='middle',
    dx=12,
    fontSize=11
).encode(
    text='Team'
)

# 4. Aggiunta delle linee medie (Quadranti)
line_x = alt.Chart(pd.DataFrame({'x': [df_plot['Avg_Shot_Length'].mean()]})).mark_rule(
    color='red', strokeDash=[3, 3], size=1
).encode(x='x')

line_y = alt.Chart(pd.DataFrame({'y': [df_plot['Avg_K_Shot'].mean()]})).mark_rule(
    color='blue', strokeDash=[3, 3], size=1
).encode(y='y')

# 5. Visualizzazione finale interattiva (Zoom + Tooltip)
final_chart = (chart + text + line_x + line_y).interactive()
final_chart.display()

alt.LayerChart(...)

In [40]:
# 1. Raccolta dati da tutti i file disponibili (Foul Committed)
results = []
TARGET_WORD = "Foul Committed"

for file in team_files:
    if os.path.exists(file):
        df = pd.read_csv(file)

        # Filtra le righe/sequenze che contengono "Foul Committed"
        foul_df = df[df["words"].astype(str).str.contains(TARGET_WORD, case=False, na=False)].copy()

        if not foul_df.empty:
            results.append({
                "Team": df["team_name"].iloc[0],
                "Foul_Rate_%": (len(foul_df) / len(df)) * 100,
                "Avg_K_Foul": foul_df["K"].mean()
            })

df_plot = pd.DataFrame(results)

# 2. Creazione dello Scatter Plot Interattivo
chart = alt.Chart(df_plot).mark_circle(size=250, opacity=0.8).encode(
    x=alt.X(
        "Foul_Rate_%",
        title="Frequenza (Foul Committed Rate %)",
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y(
        "Avg_K_Foul",
        title="Complessità Tattica (Avg K)",
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color("Team", legend=None),
    tooltip=[
        alt.Tooltip("Team", title="Squadra"),
        alt.Tooltip("Foul_Rate_%", title="Frequenza %", format=".2f"),
        alt.Tooltip("Avg_K_Foul", title="Complessità (K)", format=".4f")
    ]
).properties(
    title='Mappa del Linguaggio: "Foul Committed" — Frequenza vs Complessità',
    width=800,
    height=500
)

# 3. Etichette (nomi squadre)
text = chart.mark_text(
    align="left",
    baseline="middle",
    dx=12,
    fontSize=11
).encode(
    text="Team"
)

# 4. Linee medie (4 quadranti)
line_x = alt.Chart(pd.DataFrame({"x": [df_plot["Foul_Rate_%"].mean()]})).mark_rule(
    color="red", strokeDash=[3, 3]
).encode(x="x")

line_y = alt.Chart(pd.DataFrame({"y": [df_plot["Avg_K_Foul"].mean()]})).mark_rule(
    color="blue", strokeDash=[3, 3]
).encode(y="y")

# 5. Visualizzazione finale interattiva
final_chart = (chart + text + line_x + line_y).interactive()
final_chart.display()

alt.LayerChart(...)